In [1]:
import csv 
import string 
import re 
import pandas as pd
import string
import sklearn
import sklearn.feature_extraction
import sklearn.feature_extraction.text
import nltk
import geopandas as gpd 
import geopy 
from geopy.extra.rate_limiter import RateLimiter
from collections import OrderedDict
import ssl
import certifi
from pyspark import F
from pyspark.ml.feature import VectorAssembler, StopWordsRemover
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, count, countDistinct, max, when, split, translate
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, FloatType
from pyspark.ml.clustering import KMeans
from functools import reduce
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer, NGram
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy as np
import os
import sys
from scipy.spatial import ConvexHull

nltk.download('punkt')

nltk.download('stopwords')
sw = nltk.corpus.stopwords;
sw._LazyCorpusLoader__load();
stopwords = sw.words('english')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sarap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

ss = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

df = ss.read.options(delimiter=",", header=True, inferSchema=True).csv('03_damage-n-casualties_classify-extract/a146281_clean12.csv')

In [32]:
df2['point']=df2['point'].apply(lambda x: (str(x.latitude)+","+str(x.longitude)) if (x !='Nan' and x!=None) else 'Nan')
df3=df2.copy()
df3[['latitude', 'longitude']] = df3['point'].str.split(',', 1, expand=True)
df3[['address','latitude', 'longitude','point']]

In [3]:
    #Create dictionary and synonyms
dict_w = ['homeless', 'subway flooded', 'without power',
              'collapsed crane', 'collapsed boardwalk','research lost',
              'nuke plants','trees down','flooding','shelters','deaths','injured','missing']

syn_w = {'home': 'homeless','home destroyed':'homeless','subway':'subway flooded',
             'subway delayed': 'subway flooded' ,'subway tunnels': 'subway flooded',
             'shut down': 'without power' , 'out power': 'without power','the dark': 'without power',
             ' power outages': 'without power' ,'wo power': 'without power','in dark': 'without power',
             'loses power': 'without power','shutsdown': 'without power','electrical cables': 'without power',
             'blackouts': 'without power','lost power': 'without power','crane collapse': 'collapsed crane',
             'crane': 'collapsed crane','crane crumples': 'collapsed crane', 'crane collapses': 'collapsed crane',
             'crane dangles': 'collapsed crane','documents historical': 'research lost',
             'downed tree': 'trees down', 'tree': 'trees down' ,'trees': 'trees down',
             'flooded': 'flooding','massive surge': 'flooding','water up': 'flooding',
            'flood': 'flooding', 'water debris': 'flooding', 'sewage plant': 'flooding',
             'killed people': 'deaths','killed':'deaths','children killed': 'deaths','kills people': 'deaths',
             'dead': 'deaths','people died': 'deaths','people killed': 'deaths',
             'people dead': 'deaths'}

In [4]:
df2 = df.where(col("longitude").isNotNull())
df2 = df2.where(col("latitude").isNotNull())
df2 = df2.filter(~col('longitude').contains('Nan') & ~col('latitude').contains('Nan'))

# K-MEAN

In [5]:
#K-MEANs
K=6
df_float = df2.withColumn("latitude", df["latitude"].cast(FloatType()))\
        .withColumn("longitude", df["longitude"].cast(FloatType()))
vecAssembler = VectorAssembler(inputCols=["latitude", "longitude"], outputCol="features")
new_df = vecAssembler.transform(df_float)
kmeans = KMeans(k=K, seed=1)
model = kmeans.fit(new_df.select('features'))
transformed = model.transform(new_df)

#Inizio for
s_found = []
color = ["#FF0000","#00FF00","#0000FF","#FF8000","#7F00FF","#B39500"]
for i in range (K):
        cp=transformed.filter(transformed.prediction == i)
        #TF-IDF
        #Processing data
        tokenizer = Tokenizer(inputCol="text_no_rt", outputCol="words")
        ngram = NGram(n=2, inputCol="words", outputCol="words2")

        pipeline = Pipeline(stages=[tokenizer, ngram])
        model = pipeline.fit(cp)
        model = model.transform(cp)

        df_merged = model.select(f.concat('words', 'words2').alias("words3"))
        
        
        remover = StopWordsRemover(inputCol="words3", outputCol="words")
        df_clean = remover.transform(df_merged)
       
        
        vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
        idf = IDF(inputCol="rawFeatures", outputCol="features")
        pipeline = Pipeline(stages=[vectorizer, idf])
        model = pipeline.fit(df_clean)

        total_counts = model.transform(df_clean).select('rawFeatures').rdd \
        .map(lambda row: row['rawFeatures'].toArray()) \
        .reduce(lambda x, y: [x[i] + y[i] for i in range(len(y))])


        vocabList = model.stages[0].vocabulary
        d = {'vocabList': vocabList, 'counts': total_counts}

        representative = ss.createDataFrame(np.array(list(d.values())).T.tolist(), list(d.keys()))
        
        
        #The most representative ones are compared with a dictionary containing information that allows to identify the subevent
    #replace sinonimi con i valori del dizionario
        df_replace = representative.rdd.map(lambda x:  (syn_w[x.vocabList],x.counts)
                                            if x.vocabList in syn_w else (x.vocabList,x.counts)).toDF(["value", "count"])
     
        
        #seleziono solo quelle contenute nel vocabolario
        df_sel = df_replace.filter(f.col('value').isin(*dict_w))
       
        
        event = df_sel.collect()[0]['value']
        points = np.array(cp.select('latitude','longitude').collect())
        new_array = [tuple(row) for row in points]
        point_uniques = np.unique(new_array,axis=0).tolist()
        for point in point_uniques:
            s_found.append({
                    'event' : event, 
                    'latitude': point[0],
                    'longitude': point[1],
                    'color': color[i]
            })
        return s_found

print(s_found)

        
        
        

In [7]:
import json
with open('query16_clustering.txt', 'w') as convert_file:
     convert_file.write(json.dumps(s_found))

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53461)
Traceback (most recent call last):
  File "C:\Program Files\Python310\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Program Files\Python310\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "C:\Program Files\Python310\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Program Files\Python310\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "C:\Users\sarap\AppData\Roaming\Python\Python310\site-packages\pyspark\accumulators.py", line 281, in handle
    poll(accum_updates)
  File "C:\Users\sarap\AppData\Roaming\Python\Python310\site-packages\pyspark\accumulators.py", line 253, in poll
    if func():
  File "C:\Users\sarap\AppData\Roaming\Python\Python310\site

# TF-IDF

In [6]:
#TF-IDF
#Processing data
tokenizer = Tokenizer(inputCol="text_no_rt", outputCol="words")
ngram = NGram(n=2, inputCol="words", outputCol="words2")

pipeline = Pipeline(stages=[tokenizer, ngram])
################USARE cp INVECE DI df
model = pipeline.fit(df2)
model = model.transform(df2)

df_merged = model.select(f.concat('words', 'words2').alias("words3"))
df_merged.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|words3                                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
remover = StopWordsRemover(inputCol="words3", outputCol="words")
df_clean = remover.transform(df_merged)
#df_clean.show()

In [8]:
vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
pipeline = Pipeline(stages=[vectorizer, idf])
model = pipeline.fit(df_clean)

total_counts = model.transform(df_clean).select('rawFeatures').rdd \
        .map(lambda row: row['rawFeatures'].toArray()) \
        .reduce(lambda x, y: [x[i] + y[i] for i in range(len(y))])


vocabList = model.stages[0].vocabulary
d = {'vocabList': vocabList, 'counts': total_counts}

representative = ss.createDataFrame(np.array(list(d.values())).T.tolist(), list(d.keys()))
representative.show()

+-------------+------+
|    vocabList|counts|
+-------------+------+
|        sandy|  76.0|
|             |  57.0|
|          nyc|  44.0|
|       people|  21.0|
|        power|  20.0|
|             |  18.0|
|    people in|  18.0|
|        crane|  17.0|
|     building|  16.0|
|           rt|  16.0|
|            1|  15.0|
|          new|  14.0|
|     power in|  13.0|
|       deaths|  13.0|
|      without|  12.0|
|without power|  12.0|
|           ny|  11.0|
|        in ny|  11.0|
|       in the|  10.0|
|    deaths in|  10.0|
+-------------+------+
only showing top 20 rows



# REPLACE E CONFRONTO CON IL DIZIONARIO

In [15]:
#The most representative ones are compared with a dictionary containing information that allows to identify the sub-event
    #replace sinonimi con i valori del dizionario
df_replace = representative.rdd.map(lambda x:  (syn_w[x.vocabList],x.counts) if x.vocabList in syn_w else (x.vocabList,x.counts)).toDF(["value", "count"])
df_replace.show()

+---------------+-----+
|          value|count|
+---------------+-----+
|          sandy| 76.0|
|               | 57.0|
|            nyc| 44.0|
|         people| 21.0|
|          power| 20.0|
|               | 18.0|
|      people in| 18.0|
|collapsed crane| 17.0|
|       building| 16.0|
|             rt| 16.0|
|              1| 15.0|
|            new| 14.0|
|       power in| 13.0|
|         deaths| 13.0|
|        without| 12.0|
|  without power| 12.0|
|             ny| 11.0|
|          in ny| 11.0|
|         in the| 10.0|
|      deaths in| 10.0|
+---------------+-----+
only showing top 20 rows



In [16]:
#seleziono solo quelle contenute nel vocabolario
df_sel = df_replace.filter(f.col('value').isin(*dict_w))
df_sel.show()

+---------------+-----+
|          value|count|
+---------------+-----+
|collapsed crane| 17.0|
|         deaths| 13.0|
|  without power| 12.0|
| subway flooded|  7.0|
|       flooding|  5.0|
|collapsed crane|  4.0|
|       homeless|  4.0|
| subway flooded|  3.0|
|        injured|  3.0|
|        missing|  3.0|
|collapsed crane|  2.0|
|  research lost|  2.0|
|collapsed crane|  2.0|
|collapsed crane|  2.0|
|  without power|  1.0|
|collapsed crane|  1.0|
|       flooding|  1.0|
|  without power|  1.0|
|  without power|  1.0|
|         deaths|  1.0|
+---------------+-----+
only showing top 20 rows



# SELEZIONE DELL'EVENTO E DEL CONVEXHULL

In [19]:
event = df_sel.collect()[0]['value']
points = np.array(cp.select('latitude','longitude').collect())
print(points)
area = ConvexHull(points)
s_found['event'].append(event)
s_found['area'].append(area)


NameError: name 'df_float' is not defined